In [ ]:
# default_exp models

# Models

> Deep Learning models used to map burned areas

In [ ]:
#hide
from nbdev.export import notebook2script

In [ ]:
#export
import os, sys
from fastai.vision.all import *
import FireHR

In [ ]:
#export
def expand_filter(x, ks=3):
    with torch.no_grad():
        k5 = nn.Conv2d(1, 1, kernel_size=ks, padding=ks//2, padding_mode='reflect', bias=False)
        k5.weight.data = torch.ones(1, 1, ks, ks)/(ks*ks)
        xbuffer = k5(x[:,-1].unsqueeze(1))
        x = torch.cat([x[:,:-1], xbuffer], dim=1)
        return x
    
class ChLin(Module):
    def __init__(self, ni, nf):
        self.chlin = nn.Sequential(
            nn.Linear(ni, nf, bias=False), nn.BatchNorm1d(nf), nn.ReLU(inplace=True))
    def forward(self, x): 
        sh = x.shape
        x = x.permute(0,2,3,1).contiguous().view(sh[0]*sh[2]*sh[3], sh[1])
        x = self.chlin(x).view(sh[0],sh[2],sh[3], -1).permute(0,3,1,2).contiguous()
        return x
    
class FireHR(Module):
    def __init__(self, ni, nc):
        self.conv = ConvLayer(1, 8)
        self.chlin = nn.Sequential(ChLin(ni+8, 128), ChLin(128, 64))
        self.middleconv = nn.Sequential(ConvLayer(64, 128), ConvLayer(128, 64))
        self.finalconv = nn.Conv2d(64, nc, kernel_size=1, bias=True)
        
    def forward(self, x):
        x = torch.cat([x[:,:-1], self.conv(x[:,-1].unsqueeze(1))], dim=1)
        x = self.chlin(x)
        x = self.middleconv(x)
        return self.finalconv(x)

In [ ]:
FireHR(6,1)

FireHR(
  (conv): ConvLayer(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (chlin): Sequential(
    (0): ChLin(
      (chlin): Sequential(
        (0): Linear(in_features=14, out_features=128, bias=False)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): ChLin(
      (chlin): Sequential(
        (0): Linear(in_features=128, out_features=64, bias=False)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
  )
  (middleconv): Sequential(
    (0): ConvLayer(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )


In [ ]:
#export
_WEIGHTS = sys.modules[FireHR.__name__].__path__[0] + '/data/model512.pth'
def load_pretrained_model(weights=_WEIGHTS, ni=6, nc=1, half_precision=True, gpu=True):
    model = FireHR(ni,nc)
    st = torch.load(weights, map_location=torch.device('cpu'))
    model.load_state_dict(st['model'])
    if gpu: 
        if half_precision: model = model.half()
        if torch.cuda.is_available():
            model = model.cuda()
        else: 
            warnings.warn('GPU is not available. torch.cuda.is_available() returned False.')
    return model

In [ ]:
#local
model = load_pretrained_model()
model.conv[0].weight.device, model.conv[0].weight.type()

(device(type='cuda', index=0), 'torch.cuda.HalfTensor')

In [ ]:
#hide
notebook2script()

Converted 01_data.ipynb.
Converted 02_models.ipynb.
Converted 03_predict.ipynb.
Converted 04_cli.ipynb.
Converted index.ipynb.
